In [16]:
import torch
import optuna
import pandas as pd
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import os
import json
import random
import yaml

# Verificación de CUDA
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA disponibles:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA actual:", torch.cuda.get_device_name())

# Hiperparámetros ajustados para Detectron
HYPERPARAMETERS = {
    'max_iter': [10000, 10000],
    'max_iter_step': 1000,
    'patience': 250,
    'workers':[16, 24],
    'images_per_batch': [8, 32],
    "stepsize": [500, 2000],
    'min_max_size_train': [320, 1024],
    'checkpoint_period': -1,
    'num_trials': 20,
    'base_lr': [1e-5, 1e-2],
    'momentum': [0.7, 0.99],
    "gamma": [0.1, 0.9], # Este no no tenemos en YoloV8
    'weight_decay': [1e-5, 1e-2],
    'warmup_iters': [100, 1000],
    'warmup_momentum': [0.0, 0.95],
    'warmup_bias_lr': [1e-5, 1e-2],
    'optimizer': ['Adagrad', 'Adam', 'SGD'],
    'testing_threshold': [0.4, 0.6]
}

# Combinaciones de datasets y pesos preentrenados
combinaciones = [
    {"ruta_resultados": "models/Final/Detectron2_n_20_fix", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_n_20_pre", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/Only_Preprocessing/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_n_20_all", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/All_Data_Augmentation/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_s_20_fix", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_s_20_pre", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/Only_Preprocessing/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_s_20_all", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/All_Data_Augmentation/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_m_20_fix", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_m_20_pre", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/Only_Preprocessing/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"},
    {"ruta_resultados": "models/Final/Detectron2_m_20_all", "archivo_annotations": "/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/All_Data_Augmentation/train/annotations.json", "pre_weights": "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"}
]

# Función para obtener el número de clases desde el archivo annotations.json
def get_number_of_classes(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return len(data['categories'])

# Función para crear la estructura de directorios del experimento
def create_experiment_dir(root_dir, experiment_name):
    experiment_dir = os.path.join(root_dir, experiment_name)
    os.makedirs(experiment_dir, exist_ok=True)
    os.makedirs(os.path.join(experiment_dir, 'logs'), exist_ok=True)
    os.makedirs(os.path.join(experiment_dir, 'models'), exist_ok=True)
    return experiment_dir

# Función de entrenamiento con Detectron2
def train(cfg, max_iter, base_lr):
    cfg.SOLVER.MAX_ITER = max_iter
    cfg.SOLVER.BASE_LR = base_lr

    # Crear el entrenador y entrenar el modelo
    trainer = DefaultTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()

    # Evaluar el modelo
    evaluator = COCOEvaluator(cfg.DATASETS.TEST[0], cfg, False, output_dir=cfg.OUTPUT_DIR)
    val_loader = build_detection_test_loader(cfg, cfg.DATASETS.TEST[0])
    metrics = inference_on_dataset(trainer.model, val_loader, evaluator)

    # Verificar si 'bbox' y 'AP' están presentes en las métricas
    if 'segm' in metrics:
        ap = metrics['segm'].get('AP', 0.0)
    else:
        ap = 0.0  # Manejar el caso donde no se encuentra 'segm'

    return ap, metrics

# Función objetivo para Optuna
def objective(trial, ruta_resultados, archivo_annotations, pre_weights):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(pre_weights))
    cfg.DATASETS.TRAIN = ("my_dataset_train",)
    cfg.DATASETS.TEST = ("my_dataset_test",)
    cfg.DATALOADER.NUM_WORKERS = trial.suggest_int('workers', HYPERPARAMETERS["workers"][0], HYPERPARAMETERS["workers"][1])
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(pre_weights)
    cfg.SOLVER.IMS_PER_BATCH = trial.suggest_int('images_per_batch', HYPERPARAMETERS["images_per_batch"][0], HYPERPARAMETERS["images_per_batch"][1])
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = get_number_of_classes(archivo_annotations)

    # Generar un nombre único para el experimento basado en el número del trial
    experiment_name = f"trial_{trial.number}"
    cfg.OUTPUT_DIR = create_experiment_dir(ruta_resultados, experiment_name)
    
    # Definir los hiperparámetros para optimizar en variables para poder guardarlos luego en el json
    iter_num = trial.suggest_int('max_iter', HYPERPARAMETERS["max_iter"][0], HYPERPARAMETERS["max_iter"][1], step=HYPERPARAMETERS["max_iter_step"])
    lr = trial.suggest_loguniform('base_lr', HYPERPARAMETERS["base_lr"][0], HYPERPARAMETERS["base_lr"][1])
    gamma = trial.suggest_float('gamma', HYPERPARAMETERS["gamma"][0], HYPERPARAMETERS["gamma"][1])
    stepsize = (trial.suggest_int('stepsize', HYPERPARAMETERS["stepsize"][0], HYPERPARAMETERS["stepsize"][1]),)
    momentum = trial.suggest_float('momentum', HYPERPARAMETERS["momentum"][0], HYPERPARAMETERS["momentum"][1])
    weight_decay = trial.suggest_loguniform('weight_decay', HYPERPARAMETERS["weight_decay"][0], HYPERPARAMETERS["weight_decay"][1])
    warmup_iters= trial.suggest_int('warmup_iters', HYPERPARAMETERS["warmup_iters"][0], HYPERPARAMETERS["warmup_iters"][1])
    warmup_momentum = trial.suggest_float('warmup_momentum', HYPERPARAMETERS["warmup_momentum"][0], HYPERPARAMETERS["warmup_momentum"][1])
    testing_threshold = trial.suggest_float('testing_threshold', HYPERPARAMETERS["testing_threshold"][0], HYPERPARAMETERS["testing_threshold"][1])

    
    # Se agregan los hiperparámetros para optimizar
    cfg.SOLVER.MAX_ITER = iter_num
    cfg.SOLVER.BASE_LR = lr
    cfg.SOLVER.GAMMA = gamma
    cfg.SOLVER.STEPS = stepsize
    cfg.SOLVER.MOMENTUM = momentum
    cfg.SOLVER.WEIGHT_DECAY = weight_decay
    cfg.SOLVER.WARMUP_ITERS = warmup_iters
    cfg.SOLVER.WARMUP_FACTOR = warmup_momentum
    cfg.SOLVER.WARMUP_METHOD = "linear"
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = testing_threshold

    optimizer = trial.suggest_categorical('optimizer', HYPERPARAMETERS["optimizer"])
    if optimizer == "Adagrad":
        cfg.SOLVER.OPTIMIZER = "Adagrad"
    elif optimizer == "Adam":
        cfg.SOLVER.OPTIMIZER = "Adam"
    elif optimizer == "SGD":
        cfg.SOLVER.OPTIMIZER = "SGD"
        
    # Llamar a la función de entrenamiento con los hiperparámetros y obtener la métrica
    ap, metrics = train(cfg, iter_num, lr)
    
    # Definir la ruta para la carpeta del trial
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial.number}")
    if not os.path.exists(trial_dir):
        os.makedirs(trial_dir)

    # Extraer las métricas relevantes para guardarlas en el JSON
    segm_metrics = metrics.get("segm", {})

    AP50 = segm_metrics.get("AP50", 0)
    
    metrics_row = {
        "Trial": trial.number,
        "Dataset": archivo_annotations,
        "Feedback": "mAP50",
        "Pre-Trained_Weights": pre_weights,
        "metrics": {
            'AP': segm_metrics.get("AP", 0),
            'AP50': segm_metrics.get("AP50", 0),
            'AP75': segm_metrics.get("AP75", 0),
            'APs': segm_metrics.get("APs", 0),
            'APm': segm_metrics.get("APm", 0),
            'APl': segm_metrics.get("APl", 0)
        },
         "hyperparameters": {
            'lr': lr,
            'gamma': gamma,
            'stepsize': stepsize,
            'momentum': momentum,
            'weight_decay': weight_decay,
            'warmup_iters': warmup_iters,
            'warmup_momentum': warmup_momentum,
             'optimizer': optimizer,
            'testing_threshold': testing_threshold
        }
    }

    metrics_path = os.path.join(trial_dir, "metrics.json")
    with open(metrics_path, 'w') as outfile:
        json.dump(metrics_row, outfile)

    return AP50

# Bucle para ejecutar el entrenamiento y la optimización para cada combinación de rutas y pesos
for combinacion in combinaciones:
    ruta_resultados = combinacion["ruta_resultados"]
    archivo_annotations = combinacion["archivo_annotations"]
    pre_weights = combinacion["pre_weights"]
    
    # Definir los nombres de los datasets
    datasets = ["my_dataset_train", "my_dataset_valid", "my_dataset_test"]

    # Desregistrar los datasets si ya están registrados
    for dataset in datasets:
        if dataset in DatasetCatalog.list():
            DatasetCatalog.remove(dataset)
        if dataset in MetadataCatalog.list():
            MetadataCatalog.remove(dataset)

    # Definir los subdirectorios para train, val y test
    subdirs = ["train", "valid", "test"]
    
    print(archivo_annotations)
    dataset_root = os.path.dirname(os.path.dirname(archivo_annotations))
    
    # Registrar los datasets
    for subdir in subdirs:
        json_file = os.path.join(dataset_root, subdir, "annotations.json")
        image_dir = os.path.join(dataset_root, subdir, "images")
        register_coco_instances(f"my_dataset_{subdir}", {}, json_file, image_dir)

    # Obtener metadatos y diccionarios de los datasets
    train_metadata = MetadataCatalog.get("my_dataset_train")
    train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
    val_metadata = MetadataCatalog.get("my_dataset_valid")
    val_dataset_dicts = DatasetCatalog.get("my_dataset_valid")
    test_metadata = MetadataCatalog.get("my_dataset_test")
    test_dataset_dicts = DatasetCatalog.get("my_dataset_test")

    
    # Ejecutar la optimización con Optuna
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, ruta_resultados, archivo_annotations, pre_weights), n_trials=HYPERPARAMETERS['num_trials'])

CUDA disponible: True
Número de dispositivos CUDA disponibles: 1
Nombre del dispositivo CUDA actual: NVIDIA GeForce RTX 3090 Ti
/home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/train/annotations.json
[09/02 12:49:58 d2.data.datasets.coco]: Loaded 142 images in COCO format from /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/train/annotations.json
[09/02 12:49:58 d2.data.datasets.coco]: Loaded 48 images in COCO format from /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/valid/annotations.json
[09/02 12:49:58 d2.data.datasets.coco]: Loaded 47 images in COCO format from /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/test/annotations.json


[I 2024-09-02 12:49:58,738] A new study created in memory with name: no-name-03881496-aec8-4a93-87bc-c4d46f38223a
/tmp/ipykernel_86737/2008896220.py:135: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('base_lr', HYPERPARAMETERS["base_lr"][0], HYPERPARAMETERS["base_lr"][1])
/tmp/ipykernel_86737/2008896220.py:139: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', HYPERPARAMETERS["weight_decay"][0], HYPERPARAMETERS["weight_decay"][1])


[09/02 12:49:59 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/02 12:49:59 d2.engine.train_loop]: Starting training from iteration 0
[09/02 12:50:01 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 1.466  loss_cls: 0.6498  loss_box_reg: 0.121  loss_mask: 0.6796  loss_rpn_cls: 0.009963  loss_rpn_loc: 0.01239    time: 0.1460  last_time: 0.1675  data_time: 0.0134  last_data_time: 0.0021   lr: 8.7612e-05  max_mem: 2649M
[09/02 12:50:01 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:01 (0.1460 s / it)
[09/02 12:50:01 d2.engine.hooks]: Total training time: 0:00:01 (0:00:00 on hooks)
[09/02 12:50:01 d2.data.datasets.coco]: Loaded 47 images in COCO format from /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/Polipos265_Detectron2FIX_YoloV8/test/annotations.json
[09/02 12:50:01 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/02 12:50:01 d2.data.common]: Serializing the dataset using: <class 'detectron2.

...

[I 2024-09-02 12:51:03,990] A new study created in memory with name: no-name-b00998ca-38cd-4e07-88e5-0777019569fe
/tmp/ipykernel_86737/2008896220.py:135: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('base_lr', HYPERPARAMETERS["base_lr"][0], HYPERPARAMETERS["base_lr"][1])
/tmp/ipykernel_86737/2008896220.py:139: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', HYPERPARAMETERS["weight_decay"][0], HYPERPARAMETERS["weight_decay"][1])


[09/02 12:51:04 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[09/02 12:51:04 d2.engine.train_loop]: Starting training from iteration 0
[09/02 12:51:07 d2.utils.events]:  eta: 0:00:00  iter: 9  total_loss: 1.501  loss_cls: 0.6249  loss_box_reg: 0.1538  loss_mask: 0.6895  loss_rpn_cls: 0.01921  loss_rpn_loc: 0.006974    time: 0.1896  last_time: 0.1494  data_time: 0.0127  last_data_time: 0.0023   lr: 2.3e-05  max_mem: 3431M
[09/02 12:51:07 d2.engine.hooks]: Overall training speed: 8 iterations in 0:00:01 (0.1896 s / it)
[09/02 12:51:07 d2.engine.hooks]: Total training time: 0:00:01 (0:00:00 on hooks)
[09/02 12:51:07 d2.data.datasets.coco]: Loaded 46 images in COCO format from /home/hpc22/computer_vision_colon/TFM/Entrenamiento/datasets/DataAugmentation/All_Data_Augmentation/test/annotations.json
[09/02 12:51:07 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[09/02 12:51:07 d2.data.common]: Serializing the dataset using: <class 'detect

[I 2024-09-02 12:51:17,579] Trial 0 finished with value: 0.5252955263877885 and parameters: {'workers': 2, 'images_per_batch': 2, 'max_iter': 10, 'base_lr': 2.299980941795077e-05, 'gamma': 0.6071324479460453, 'stepsize': 954, 'momentum': 0.7568675450628453, 'weight_decay': 0.0006959482801866375, 'warmup_iters': 1, 'warmup_momentum': 0.6814938649330338, 'testing_threshold': 0.4478994187908241, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.5252955263877885.
